<a href="https://colab.research.google.com/github/kdovin/project_Lovely-G.4_none/blob/main/XGBoost%2BLightGBM%2BCatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# < 데이터 불러오기 >

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 기본 한글 폰트로 설정
import matplotlib.font_manager as fm  # 폰트 관리


!apt-get update -qq         # apt-get 패키지 설치 명령어, -qq : 에러외 메세지 숨기기
!apt-get install fonts-nanum* -qq #나눔글꼴 설치

fe = fm.FontEntry(fname=r'C:/Windows/Fonts/NanumGothic.ttf', name='NanumGothic') #파일 저장되어있는 경로와 이름 설정
fm.fontManager.ttflist.insert(0, fe)  # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumGothic'}) #폰트설정
plt.rcParams['font.family'] = 'NanumGothic'

# 음수 부호까지 제대로 표시되도록 설정 추가
plt.rcParams['axes.unicode_minus'] = False

# 경고 메시지 무시
import random, warnings
warnings.filterwarnings("ignore")

Selecting previously unselected package fonts-nanum.
(Reading database ... 121752 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-3_all.deb ...
Unpacking fonts-nanum-coding (2.5-3) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-7_all.deb ...
Unpacking fonts-nanum-eco (1.000-7) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20200506-1_all.deb ...
Unpacking fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum-extra (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Setting up fonts-nanum-coding (2.5-3) ...
Setting up fonts-nanum-eco (1.000-7) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [2]:
#데이터 불러오기
train_df = pd.read_csv('/content/drive/MyDrive/데이터/주택가격/train.csv') #모델 학습 파일
test_df = pd.read_csv('/content/drive/MyDrive/데이터/주택가격/test.csv') #모델 시험지 파일
submission = pd.read_csv('/content/drive/MyDrive/데이터/주택가격/sample_submission.csv') #답안지 파일

# < 이상치 처리 >

In [3]:
columns_to_replace = ['LotFrontage', 'BsmtFinSF1', 'TotalBsmtSF', 'EnclosedPorch', 'MiscVal']  # 대체하려는 변수들의 리스트
values_to_replace = [300, 5000, 5000, 500, 8000]  # 대체하려는 값들의 리스트

for column, value in zip(columns_to_replace, values_to_replace):
    median_value = train_df.loc[train_df[column] < value, column].median()  # 대체할 값(value)이 아닌 값들의 중앙값 계산
    train_df.loc[train_df[column] > value, column] = median_value  # 대체할 값(value)을 중앙값으로 대체

In [4]:
columns2_to_replace = ['BsmtFinSF1', 'TotalBsmtSF', 'WoodDeckSF', 'EnclosedPorch', 'MiscVal', '1stFlrSF', 'LowQualFinSF', 'GarageYrBlt']  # 대체하려는 변수들의 리스트
values2_to_replace = [3500,4000,1200,800,10000,4000,1000,2150]  # 대체하려는 값들의 리스트

for column, value in zip(columns2_to_replace, values2_to_replace):
    median_value = test_df.loc[test_df[column] < value, column].median()  # 대체할 값(value)이 아닌 값들의 중앙값 계산
    test_df.loc[test_df[column] > value, column] = median_value  # 대체할 값(value)을 중앙값으로 대체

# < PCA 추가 이상치 처리 >

In [5]:
from sklearn.decomposition import PCA

# house_df 생성
house_df = pd.concat([train_df, test_df], ignore_index=True, sort=False)

# train_df에서 SalePrice가 null이 아닌 행의 인덱스 추출
tr_idx = house_df['SalePrice'].notnull()

# PCA 적용 함수 정의
def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # PCA 적용
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # DataFrame으로 변환
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Loadings 생성
    loadings = pd.DataFrame(
        pca.components_.T,
        columns=component_names,
        index=X.columns,
    )
    return pca, X_pca, loadings

# IQR을 사용하여 이상치 감지 함수 정의
def outlier_iqr(data):
    q1, q3 = np.nanpercentile(data, [25, 75])
    iqr = q3 - q1
    lower = q1 - (iqr * 2)
    upper = q3 + (iqr * 2)
    return np.where((data > upper) | (data < lower))[0]

# 이상치를 찾을 feature 선택
features = ["GarageArea", "OverallQual", "TotalBsmtSF", "GrLivArea"]

# 이상치 제거 전에 데이터 복사
house_copy = house_df[tr_idx].copy()
y_copy = house_copy.pop("SalePrice")
X_copy = house_copy.loc[:, features]

# PCA 적용
pca, X_pca, loadings = apply_pca(X_copy)

# Plotly를 사용하여 PCA 히스토그램 생성
import plotly.express as px
fig = px.histogram(X_pca[['PC1', 'PC2']].melt(), color="variable",
                   marginal="box",
                   barmode="overlay",
                   histnorm='density'
                  )
fig.update_layout(
    title_font_color="white",
    legend_title_font_color="yellow",
    title={
        'text': "PCA Histogram",
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    paper_bgcolor="black",
    plot_bgcolor='black',
    font_color="white",
)

# PC1에 대한 이상치 감지
pc1_outlier_idx = outlier_iqr(X_pca['PC1'])
df_temp_pc1 = house_df.loc[:, ["SalePrice", "Neighborhood", "SaleCondition"] + features]
df_temp_pc1.iloc[pc1_outlier_idx,:].style.set_properties(**{'background-color': 'Grey', 'color': 'white', 'border-color': 'darkblack'})

# PC2에 대한 이상치 감지
pc2_outlier_idx = outlier_iqr(X_pca['PC2'])
df_temp_pc2 = house_df.loc[:, ["SalePrice", "Neighborhood", "SaleCondition"] + features]
df_temp_pc2.iloc[pc2_outlier_idx,:].style.set_properties(**{'background-color': 'Grey', 'color': 'white', 'border-color': 'darkblack'})

# 이상치 제거
house_df.drop(pc1_outlier_idx, axis=0, inplace=True, errors='ignore')
house_df.drop(pc2_outlier_idx, axis=0, inplace=True, errors='ignore')


# < PCA 추가 이상치 처리 >

In [6]:
# 왜도 첨도 확인!
# 왜도 1이상, 첨도 3이상이면 로그변환 필요! (딘, 무조건 필요한 건 아님)

for col in features:
    print('{:15}'.format(col),
          'Skewness: {:05.2f}'.format(house_df[col].skew()) ,
          '   ' ,
          'Kurtosis: {:06.2f}'.format(house_df[col].kurt())
         )

GarageArea      Skewness: 00.21     Kurtosis: 000.87
OverallQual     Skewness: 00.18     Kurtosis: -00.01
TotalBsmtSF     Skewness: 00.28     Kurtosis: 000.75
GrLivArea       Skewness: 00.97     Kurtosis: 001.86


In [7]:
house_df["LotArea"] = np.log1p(house_df["LotArea"])
house_df["LowQualFinSF"] = np.log1p(house_df["LowQualFinSF"])
house_df["PoolArea"] = np.log1p(house_df["PoolArea"])
house_df["3SsnPorch"] = np.log1p(house_df["3SsnPorch"])
house_df["MiscVal"] = np.log1p(house_df["MiscVal"])

In [8]:
# 지수변환
house_df['TotalBsmtSF'] = house_df['TotalBsmtSF'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
house_df['2ndFlrSF'] = house_df['2ndFlrSF'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)
house_df['GarageArea'] = house_df['GarageArea'].apply(lambda x: np.exp(6) if x <= 0.0 else x)
house_df['GarageCars'] = house_df['GarageCars'].apply(lambda x: 0 if x <= 0.0 else x)
house_df['LotFrontage'] = house_df['LotFrontage'].apply(lambda x: np.exp(4.2) if x <= 0.0 else x)
house_df['MasVnrArea'] = house_df['MasVnrArea'].apply(lambda x: np.exp(4) if x <= 0.0 else x)
house_df['BsmtFinSF1'] = house_df['BsmtFinSF1'].apply(lambda x: np.exp(6.5) if x <= 0.0 else x)

In [9]:
numerical_features_log = [feature for feature in house_df.columns if house_df[feature].dtypes != 'O']

print('Number of numerical variables: ', len(numerical_features_log))

Number of numerical variables:  38


In [10]:
# 왜도 첨도 확인!
# 왜도 1이상, 첨도 3이상이면 로그변환 필요! (딘, 무조건 필요한 건 아님)

for col in features:
    print('{:15}'.format(col),
          'Skewness: {:05.2f}'.format(house_df[col].skew()) ,
          '   ' ,
          'Kurtosis: {:06.2f}'.format(house_df[col].kurt())
         )

GarageArea      Skewness: 00.90     Kurtosis: 001.32
OverallQual     Skewness: 00.18     Kurtosis: -00.01
TotalBsmtSF     Skewness: 00.64     Kurtosis: 000.45
GrLivArea       Skewness: 00.97     Kurtosis: 001.86


In [11]:
house_df["SalePrice"] = np.log1p(house_df["SalePrice"])

# < 파생변수 >

In [13]:
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 객체 생성
label_encoder = LabelEncoder()

# ExterQual, ExterCond, BsmtQual, BsmtCond, BsmtFinType1, BsmtFinType2, GarageQual, GarageCond 열을 숫자형으로 변환
columns_to_encode = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinType1', 'BsmtFinType2', 'GarageQual', 'GarageCond']
for column in columns_to_encode:
    house_df[column + '_num'] = label_encoder.fit_transform(house_df[column])

# TotalExtQual 열을 생성하고 숫자형으로 변환
quality_mapping = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1}  # 각 품질 등급을 숫자로 매핑
house_df['TotalExtQual'] = house_df['ExterQual'].map(quality_mapping) + house_df['ExterCond'].map(quality_mapping)

# 연산에 필요한 새로운 열들을 생성
house_df['TotalSF'] = house_df['BsmtFinSF1'] + house_df['BsmtFinSF2'] + house_df['1stFlrSF'] + house_df['2ndFlrSF']
house_df['TotalBathrooms'] = house_df['FullBath'] + (0.5 * house_df['HalfBath']) + house_df['BsmtFullBath'] + (0.5 * house_df['BsmtHalfBath'])
house_df['YearBlRm'] = house_df['YearBuilt'] + house_df['YearRemodAdd']
house_df['TotalPorchSF'] = house_df['OpenPorchSF'] + house_df['3SsnPorch'] + house_df['EnclosedPorch'] + house_df['ScreenPorch'] + house_df['WoodDeckSF']
house_df['TotalBsmQual'] = house_df['BsmtQual_num'] + house_df['BsmtCond_num'] + house_df['BsmtFinType1_num'] + house_df['BsmtFinType2_num']
house_df['TotalGrgQual'] = house_df['GarageQual_num'] + house_df['GarageCond_num']

# QualPorch 열 생성
house_df['QualPorch'] = house_df['TotalExtQual'] * house_df['TotalPorchSF']

In [14]:
house_df['YrBltAndRemod']=house_df['YearBuilt']+house_df['YearRemodAdd']

house_df['Total_sqr_footage'] = (house_df['BsmtFinSF1'] + house_df['BsmtFinSF2'] +
                                 house_df['1stFlrSF'] + house_df['2ndFlrSF'])

In [15]:
# simplified house_df
house_df['haspool'] = house_df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
house_df['has2ndfloor'] = house_df['2ndFlrSF'].apply(lambda x: 1 if x > 0 else 0)
house_df['hasgarage'] = house_df['GarageArea'].apply(lambda x: 1 if x > 0 else 0)
house_df['hasbsmt'] = house_df['TotalBsmtSF'].apply(lambda x: 1 if x > 0 else 0)
house_df['hasfireplace'] = house_df['Fireplaces'].apply(lambda x: 1 if x > 0 else 0)

In [16]:
house_df['HasWoodDeck'] = (house_df['WoodDeckSF'] == 0) * 1
house_df['HasOpenPorch'] = (house_df['OpenPorchSF'] == 0) * 1
house_df['HasEnclosedPorch'] = (house_df['EnclosedPorch'] == 0) * 1
house_df['Has3SsnPorch'] = (house_df['3SsnPorch'] == 0) * 1
house_df['HasScreenPorch'] = (house_df['ScreenPorch'] == 0) * 1
house_df['YearsSinceRemodel'] = house_df['YrSold'].astype(int) - house_df['YearRemodAdd'].astype(int)
house_df['Total_Home_Quality'] = house_df['OverallQual'] + house_df['OverallCond']
house_df['Total_SF'] = house_df['TotalBsmtSF'] + house_df['1stFlrSF'] + house_df['2ndFlrSF']

In [17]:
house_df['has2ndfloor']

0       1
1       0
2       1
3       1
4       1
       ..
2914    0
2915    0
2916    0
2917    0
2918    1
Name: has2ndfloor, Length: 2907, dtype: int64

In [18]:
numerical_features2 = [feature for feature in house_df.columns if house_df[feature].dtypes != 'O']

print('Number of numerical variables: ', len(numerical_features2))

Number of numerical variables:  69


# < 인코딩 >

In [19]:
# 범주형 변수들의 리스트 반환
categorical_features = house_df.select_dtypes(include=['object']).columns.tolist()
print("범주형 변수들의 리스트:", categorical_features)

범주형 변수들의 리스트: ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [20]:
# 범주형 변수들을 원-핫 인코딩하여 모델에 사용할 수 있도록 변환
house_df_encoded = pd.get_dummies(house_df, columns=categorical_features)

# < 모델링 >

In [21]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.8 MB/s eta 0:00:00


In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.model_selection import GridSearchCV

# 분할
y = house_df_encoded[tr_idx].pop('SalePrice')
X = house_df_encoded[tr_idx].drop('SalePrice',axis=1,errors='ignore')
X_sub = house_df_encoded[~tr_idx].drop('SalePrice',axis=1,errors='ignore')
print('X', X.shape, 'y', y.shape, 'X_sub')

# XGBoost 모델 정의
xgb_regressor = XGBRegressor()

# LightGBM 모델 정의
lgbm_regressor = LGBMRegressor()

# CatBoost 모델 정의
catboost_regressor = CatBoostRegressor(verbose=0)

# 앙상블 모델 정의
ensemble_regressor = VotingRegressor([('xgb', xgb_regressor), ('lgbm', lgbm_regressor), ('catboost', catboost_regressor)])

# 그리드 서치를 통해 최적의 하이퍼파라미터 찾기
param_grid = {
    'xgb__max_depth': [3, 5, 7],
    'lgbm__max_depth': [3, 5, 7],
    'catboost__depth': [3, 5, 7]
}
grid_search = GridSearchCV(ensemble_regressor, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)
grid_search.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("Best parameters found: ", grid_search.best_params_)

# 최적의 모델로 재학습
best_ensemble_regressor = grid_search.best_estimator_
best_ensemble_regressor.fit(X, y)

# 학습 데이터에 대한 예측값 생성
train_predictions = best_ensemble_regressor.predict(X)

# RMSE 계산
train_rmse = mean_squared_error(y, train_predictions, squared=False)
print("Training RMSE: ", train_rmse)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [ ]:
# 테스트 데이터에 대한 예측값 생성
test_predictions = best_ensemble_regressor.predict(X_sub)

# submission 데이터프레임에 예측값 삽입
submission['SalePrice'] = np.expm1(test_predictions)  # 로그 스케일로 변환된 SalePrice를 다시 원래 스케일로 변환

# CSV 파일로 저장
submission.to_csv('XGBoost+LightGBM+CatBoost.csv', index=False)